
---

# Silhouette masking with OpenCV

---

One component of the art/tech project I'm working on for the Dec 2019 exhibition requires creating silhouette masks of people moving around a webcam. This notebook explores different ways to do that.

---

## Setup

---

In [1]:
import os

import cv2 as cv
import imutils
import numpy as np

---

## Detection via background subtraction

---

One possibility for silhouette masking is _background subtraction_ - using motion and color changes in a video feed to find static background regions vs. dynamic foreground regions OpenCV has multiple background subtraction methods.

#### Background subtraction for a webcam

Press 'Q' to quit.

In [ ]:
cap = cv.VideoCapture(0)
fgbg = cv.createBackgroundSubtractorKNN()

while(1):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    fgmask = cv.medianBlur(fgmask, 11)
    fgmask = cv.GaussianBlur(fgmask, (5, 5), 0)
    cv.imshow('frame', frame)
    cv.imshow('mask', fgmask)
    if cv.waitKey(1) & 0xFF == ord('q'):
        print(frame.min(), frame.max())
        break
        
cap.release()
cv.destroyAllWindows()

#### Comparing subtraction algorithms on a test video

In [2]:
def background_subtraction_mosaic(
    video_file,
    save_file,
    subtractors,
    median_blur_size,
    gaussian_blur_size,
    scale,
    spacing,
    show_current_frame=False):
    """Run a background subtraction configuration on a source
    video file. Save the output to another video.
    """
    # Create a VideoCapture from a video file
    cap = cv.VideoCapture(video_file)
    # Get video file attributes for creating the output video
    n_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    fps = int(cap.get(cv.CAP_PROP_FPS))
    
    # Codec for the VideoWriter
    fourcc = cv.VideoWriter_fourcc(*'X264')
    # Create a VideoWriter
    w_each = int(scale * width)
    h_each = int(scale * height)
    ncols = len(subtractors) + 1
    nrows = max([len(s) for s in subtractors])
    w = w_each * ncols + spacing * (ncols - 1)
    h = h_each * nrows + spacing * (nrows - 1)
    vid_out_size = (w, h)
    vid_out = cv.VideoWriter(
        save_file, 
        fourcc, 
        fps, 
        vid_out_size,
        True)
    
    # background is off-white
    frame_out = 235 * np.ones((h, w, 3)).astype(np.uint8)
    for _ in range(n_frames):
        ret, frame = cap.read()
        if show_current_frame:
            cv.imshow('frame', frame)
            cv.waitKey(1)
        fresized = imutils.resize(frame, height=h_each, width=w_each)
        frame_out[:h_each, :w_each, :] = fresized
        for x, col in enumerate(subtractors):
            x0 = (x + 1) * (w_each + spacing) 
            x1 = x0 + w_each
            for y, subtractor in enumerate(col):
                y0 = y * (h_each + spacing)
                y1 = y0 + h_each
                mask = subtractor.apply(frame)
                if median_blur_size > 0:
                    mask = cv.medianBlur(mask, median_blur_size)
                if gaussian_blur_size > 0:
                    shape = (gaussian_blur_size, gaussian_blur_size)
                    mask = cv.GaussianBlur(mask, shape, 0)
                
                mresized = imutils.resize(
                    mask, 
                    height=h_each,
                    width=w_each)
                frame_out[y0:y1, x0:x1, :] = np.stack(
                    [mresized]*3,
                    axis=-1)
        vid_out.write(frame_out)
        
    cap.release()
    vid_out.release()
    cv.destroyAllWindows()
    pass

#### Define some subtraction algorithm configs to test

In [3]:
save_dir = 'masks'
os.makedirs(save_dir, exist_ok=True)

median_value = 11
gaussian_value = 5

subtractors = [
    [
        cv.createBackgroundSubtractorKNN(500, 400, True),
        cv.createBackgroundSubtractorKNN(200, 400, True),
        cv.createBackgroundSubtractorKNN(200, 1000, True),
        cv.createBackgroundSubtractorKNN(500, 200, True),
        cv.createBackgroundSubtractorKNN(500, 1000, True),
        cv.createBackgroundSubtractorKNN(500, 400, False)
    ],
    
    [
        cv.createBackgroundSubtractorMOG2(500, 16, True),
        cv.createBackgroundSubtractorMOG2(200, 16, True),
        cv.createBackgroundSubtractorMOG2(200, 32, True),
        cv.createBackgroundSubtractorMOG2(500, 10, True),
        cv.createBackgroundSubtractorMOG2(500, 32, True),
        cv.createBackgroundSubtractorMOG2(500, 16, False)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorCNT(15, True, 900),
        cv.bgsegm.createBackgroundSubtractorCNT(15, True, 400),
        cv.bgsegm.createBackgroundSubtractorCNT(5, True, 900),
        cv.bgsegm.createBackgroundSubtractorCNT(45, True, 900),
        cv.bgsegm.createBackgroundSubtractorCNT(5, True, 400),
        cv.bgsegm.createBackgroundSubtractorCNT(45, True, 400)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.8),
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.9),
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.95),
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.7),
        cv.bgsegm.createBackgroundSubtractorGMG(45, 0.6),
        cv.bgsegm.createBackgroundSubtractorGMG(20, 0.8)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.003, propagationRate=0.01),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.003, propagationRate=0.025),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.006, propagationRate=0.01),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.0015, propagationRate=0.01),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.006, propagationRate=0.025),
        cv.bgsegm.createBackgroundSubtractorGSOC(replaceRate=0.0015, propagationRate=0.025)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0004, noiseRemovalThresholdFacFG=0.0008),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0004, noiseRemovalThresholdFacFG=0.00016),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0002, noiseRemovalThresholdFacFG=0.0004),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0008, noiseRemovalThresholdFacFG=0.0004),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0002, noiseRemovalThresholdFacFG=0.0016),
        cv.bgsegm.createBackgroundSubtractorLSBP(noiseRemovalThresholdFacBG=0.0008, noiseRemovalThresholdFacFG=0.0016)
    ],
    
    [
        cv.bgsegm.createBackgroundSubtractorMOG(200, 5, 0.7),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 3, 0.7),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 7, 0.7),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 5, 0.55),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 5, 0.85),
        cv.bgsegm.createBackgroundSubtractorMOG(200, 7, 0.85)
    ]
]

background_subtraction_mosaic(
    'test.mp4',
    os.path.join(save_dir, 'mosaic1013.mp4'),
    subtractors,
    median_value,
    gaussian_value,
    0.37,
    2,
    True)